In [24]:
# takes a driver to target, path to directory of driver.csv's, and returns an RDD labeled with
# (driverID, tripID),(x,y,step,label), where a label 1 is from an actual trip, and label 0 is from
# a trip randomly sampled from other drivers
def labelRDDs(targ_driv, path, K=200):
    """
    :param targ_driv: string, driver we want to create positive and negative labeled data for
    :param path: string, path to directory where driver.csvs are stored
    :param K: int, number of negative (manufactured) trips to sample
    :return: RDD with key, value tuple where key is (driverID, tripID) and value is (x,y,step,label)
    """
    try:
        newfile = path + '/tempscsvs.csv'
        target = sc.textFile(path + '/' + 'driver_' + targ_driv + '.csv') #load target driver's data
        target2 = target.map(lambda x: x.split(',')) #convert from string to list of strings
        positives = processRDD(target2, 1.0) #label target driver's RDD
        driv_lis = get_drivers(path) #get python list of all possible drivers to sample from
        sampdriv, samptrip = random_samples(targ_driv, driv_lis, K) #generate random samples of drivers and tripIDs
        samples = sample_data(path, sampdriv, samptrip, newfile) #generate RDD of random samples
        samplesRDD = ID_Data(targ_driv, samples, K) #relabel samples to look like target driver's trips
        negatives = processRDD(samplesRDD, 0.0) #label samples
        finalRDD = positives.union(negatives).cache() #join target driver and samples together
        return finalRDD
    except Exception as e:
        print e

In [25]:
KKK = labelRDDs('1', path)

[Errno 2] No such file or directory: '/Users/coolguy/Desktop/ML2Project/Data/processed_data/sample_drivers/driver_.csv'
'NoneType' object has no attribute 'map'
'NoneType' object has no attribute 'map'
'NoneType' object has no attribute '_jrdd_deserializer'


In [10]:
A

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '3',
 '30']

In [8]:
#!/usr/bin/env python

# this is a pyspark file, not python
# functions in this file are meant to read the entire telematic data, and given one driver
# produce a new RDD with 400 trips: 200 from original driver, labeled as '1' and sampled
# from other drivers labeled as '0'

# note: resulting RDD will have 400 trips, the rows of the RDD will be individual observations
# from each of those trips - therefore, there will be more then 400 rows

# note: these functions expect the data to be stored as multiple CSV files, one for each driver,
# stored in the same directory, with the format: 'x, y, driverID, tripID, step'
# and named as '1.csv', '2.csv', etc.

# note: these functions will output in the format: 'x, y, driverID, tripID, step, label'

# import modules
import numpy as np
import os as os
import re as re
import fileinput

__author__ = "Su-Young Hong AKA Da Masta Killa AKA Synth Pop Rocks of Locks AKA Intergalactic Chilympian"
__status__ = "Prototype"

# read directory of files and return a list of all driverIDs from csv's insid directory
def get_drivers(dirpath):
    """
    :param dirpath: string, path to directory containing driver csv's
    :return: list, contains all driverIDs as strings
    """
    try:
        allfiles = os.listdir(dirpath)
        drivers = [re.sub(r'[^0-9]', '', i) for i in allfiles]
        drivers.remove('')
        return drivers
    except Exception as e:
        print e

In [ ]:
# produces random samples of driverIDs and tripIDs in two separate lists
def random_samples(targ_driv, driv_list, K=200):
    """
    :param targ_driv: str, driverID we want to make false trips for
    :param driv_list: list, list of all drivers, produced by get_drivers()
    :param K: number of trips we want to make for targ_driv
    :return: tuple of lists, first list is random driverIDs, second list is list of tripIDs, both are strings
    """
    try:
        driv_list.remove(targ_driv) #removes the target driver from list of drivers to sample from
        drivers = np.random.choice(driv_list, K, True)
        trips = np.random.choice(np.arange(1,K+1).astype(str), K, True)
        return (drivers, trips)
    except Exception as e:
        print e

In [11]:
B = random_samples(driver, A)

In [12]:
B


(array(['15', '18', '14', '20', '19', '16', '18', '14', '19', '30', '24',
        '23', '12', '3', '28', '30', '18', '25', '22', '21', '27', '3',
        '26', '23', '12', '19', '10', '18', '17', '14', '26', '26', '10',
        '22', '25', '14', '2', '12', '20', '19', '12', '22', '22', '15',
        '23', '28', '13', '10', '11', '14', '11', '19', '20', '30', '17',
        '16', '26', '20', '16', '16', '25', '13', '12', '19', '17', '11',
        '26', '14', '30', '3', '25', '19', '25', '14', '3', '27', '18',
        '13', '13', '30', '2', '2', '16', '25', '13', '21', '12', '24',
        '28', '25', '30', '26', '25', '16', '12', '28', '21', '21', '16',
        '19', '24', '20', '25', '10', '21', '19', '23', '23', '23', '23',
        '23', '12', '23', '19', '16', '15', '13', '22', '11', '12', '28',
        '30', '23', '14', '25', '10', '13', '23', '28', '15', '16', '11',
        '16', '22', '2', '10', '10', '10', '25', '10', '23', '11', '23',
        '20', '28', '21', '26', '15', '22', '2

In [13]:
# reads directory of files and returns RDD of observations from trips in the sample (driverID, tripID combo)
# NOTE: this function is VERY SLOW, it is what slows the entire workflow down
def sample_data(path, driverIDs, tripIDs, newfile):
    """
    :param path: string, path to directory containing driver.csv's
    :param driverIDs: list, list of randomly sampled driverIDs as strings, produced by random_sample()
    :param tripIDs: list, list of randomly sampled tripIDs as strings, produced by random_samples()
        NOTE: the above two zip into a list of (driverID, tripID) tuples, with each tuple being a single item in the
        sample
    :return: RDD, contains only observations from the sample
    """
    try:
        combos = zip(driverIDs, tripIDs)
        samplefiles = [path + '/' + 'driver_' + i + '.csv' for i in set(driverIDs)]
        fpw = open(newfile, 'w')
        fpr = fileinput.input(samplefiles)
        for line in fpr:
            fpw.write(line)
        fpw.close()
        fpr.close()
        RDD = sc.textFile(newfile)   #### NOTE: with large num. files, might need to set num. partitions
        RDDsplit = RDD.map(lambda x: x.split(','))
        RDDsamples = RDDsplit.filter(lambda x: (x[2],x[3]) in combos)
        RDDsamples.cache()
        return RDDsamples
    except Exception as e:
        print e

In [14]:
newthingy = path + '/tempscsvs.csv'

In [16]:
C = sample_data(path, B[0],B[1], newthingy)

In [17]:
C.count()

126389

In [ ]:
# takes RDD of samples and assigns new driverID and tripID to observations in a new RDD
def ID_Data(targ_driver, RDD, K = 200):
    """
    :param targ_driver: string, target driver we used to generate samples
    :param RDD: RDD, trip data RDD produced by sample_data(), format will be original form (x,y,driverID,tripID,step)
    :param K: int, number of trips we sampled
    :return: RDD, in original format, but with driverID and tripID changed to look like new observations of the target
    driver
    """
    try:
        newID1 = [targ_driver] * K
        newID2 = np.arange(200, 201+K).astype(str)
        newID = zip(newID1, newID2)
        oldID = RDD.map(lambda x: (x[2],x[3])).distinct().collect()
        glossary = sc.parallelize(zip(oldID, newID))
        newRDD = RDD.map(lambda x: ((x[2],x[3]), ([x[0],x[1],x[4]]))).join(glossary)
        newID_RDD = newRDD.map(lambda x: (x[1][0][0], x[1][0][1], x[1][1][0], x[1][1][1], x[1][0][2]))
        return newID_RDD
    except Exception as e:
        print e

In [20]:
D = ID_Data('1', C)

In [21]:
D.first()

(u'0.0', u'0.0', '1', '221', u'0')

In [ ]:
# takes RDD in original form and converts it into key-value tuple with values being x,y,step,label
def processRDD(RDD, label):
    """
    :param RDD: RDD in original format (x,y,driverID,tripID,step)
    :param label: category of observation, 1 for positive, 0 for negative
    # note, not sure if it needs to be int or float
    :return: RDD, RDD returned in new key/value format: (driverID, tripID), (x, y, step, label)
    # note, x, y, step, and label will be floats
    """
    try:
        newRDD = RDD.map(lambda x: ((x[2],x[3]),(float(x[0]),float(x[1]),float(x[4]),label)))
        return newRDD
    except Exception as e:
        print e

In [ ]:
processRDD()

In [4]:

path = '/Users/coolguy/Desktop/ML2Project/Data/processed_data/sample_drivers'
driver = '1'
newRDD = labelRDDs(driver, path)


[Errno 2] No such file or directory: ''


In [7]:
get_drivers(path)

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '3',
 '30']